总而言之，Speck 系列开发版使用三个库进行编程：Tonic, Sinabs, 和Samna。

Tonic(https://tonic.readthedocs.io/en/latest/index.html)是数据预处理库，提供公开的基于事件的视觉和音频数据集以及事件转换。

Sinabs(https://synsense.gitlab.io/sinabs-dynapcnn/)用于软件仿真，这里的主要作用是将神经网络转换为Speck可接受的格式。

Samna(https://synsense-sys-int.gitlab.io/samna/devkits/speckSeries/examples/speck2f_proximity_detection.html)用于软硬件交互，部署神经网络到speck硬件上并控制pin的输入/输出。

这里先从调用DVS相机开始讲。

## 监控 DVS 事件

通过修改硬件配置对象，可以选择是否监控集成 DVS 生成的事件。

如["Visualize DVS Input"](/getting_started/notebooks/visualize_speck_dvs_input.ipynb)所述，硬件配置对象有 2 个相关属性：

- samna.speckxx.configuration.SpeckConfiguration.dvs_layer.raw_monior_enable"。如果设置为 "true"，用户就可以监控由 DVS 生成的原始事件，监控事件的类型为 "samna.speckxx.event.DvsEvent"。原始 DVS 事件意味着它们直接来自 DVS 阵列，不会受到裁剪、镜像、过滤等任何预处理的影响。更多详情，请参阅 [samna documentation.](https://synsense-sys-int.gitlab.io/samna/reference/speck2f/event/index.html#samna.speck2f.event.DvsEvent)。

- samna.speckxx.configuration.SpeckConfiguration.dvs_layer.monior_enable"。如果设置为 "true"，用户就可以监控预处理的 DVS 事件。监控事件的类型将是 `samna.speckxx.event.Spike`，并且必须带有属性 `.layer = 13`，即所有来自 #13 层的 `samna.speckxx.event.Spike`都是 DVS 层的输出事件。裁剪、镜像、过滤等预处理操作会影响这些事件。

其中 `speckxx` 表示不同的 Speck devkit 板。

DynapcnnVisualizer "正在监控 "samna.speckxx.event.Spike"。因此，我们可以根据不同的硬件配置更改可视化器显示的结果。

## Merge Polarity

Speck的DVS事件支持两种极性，在实际使用中可以选择合并极性。

将 2 个极性合并为 1 个极性的最简单方法是：

input_shape将参数设置为初始化.input_shape=(1, XX, XX)DynapcnnNetwork

另一种方法是：

在将硬件配置应用于 devkit 之前手动修改硬件配置。samna.speckxx.configuration.SpeckConfiguration.dvs_layer.merge = True


In [33]:
from torch import nn
from sinabs.layers import IAFSqueeze
from sinabs.backend.dynapcnn.dynapcnn_visualizer import DynapcnnVisualizer
from sinabs.backend.dynapcnn import DynapcnnNetwork


# create a dummy snn for DynapcnnNetwork initialization
snn = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# init DynapcnnNetwork
input_shape = (1, 128, 128)
dynapcnn = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True)

# deploy to speck devkit, use a different name if you're using a different version of the devkit
devkit_name = "speck2fmodule"
dynapcnn.to(device=devkit_name, monitor_layers=["dvs", -1])

Network is valid


DynapcnnNetwork(
  (sequence): Sequential(
    (0): DVSLayer(
      (pool_layer): SumPool2d(norm_type=1, kernel_size=(1, 1), stride=None, ceil_mode=False)
      (crop_layer): Crop2d((0, 128), (0, 128))
      (flip_layer): FlipDims()
    )
    (1): DynapcnnLayer(
      (conv_layer): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (spk_layer): IAFSqueeze(spike_threshold=398.0, min_v_mem=-398.0, batch_size=1, num_timesteps=-1)
    )
  )
)

接下来，我们利用 "DynapcnnVisualizer "来可视化 DVS 事件。可视化器将显示 3 个子窗口
- Dvs 图，显示 DVS 事件。
- 尖峰计数图，显示 0 号层的输出尖峰曲线数。
- 功率测量图，显示 "io"、"ram "和 "logic "的功耗。

**运行notebook时，请不要关闭visulizer。**

In [34]:
visualizer = DynapcnnVisualizer(
    window_scale=(4, 8),
    dvs_shape=(128, 128),
    add_power_monitor_plot=True,
    spike_collection_interval=1000,  # milii-second
)

visualizer.connect(dynapcnn_network=dynapcnn)

Connecting: Please wait until the JIT compilation is done, this might take a while. You will get notified on completion.
Set up completed!


## 不合并极性

设置 `input_shape = (2, xx, xx)`

In [35]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_bi_polarity = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_bi_polarity)

Network is valid


## 池化

利用 DVS 层的池化功能的最简单方法是：

添加一个 `nn.AvgPool2d` 或 `sinabs.layers.SumPool2d` 作为序列模型的第一层。

另一种方法是

手动修改硬件配置 `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.pooling.x 和 .y` 。默认的`.x`和`.y`是1。`.x`和`.y`的可用数字是：  **{1, 2, 4}**

In [43]:
snn = nn.Sequential(
    # this pooling operation will be executed in the DVS Layer when deployed to the hardware
    nn.AvgPool2d(kernel_size=(2, 2)),
    # the input of the 1st DynapCNN Core will be (1, 64, 64)
    nn.Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (1, 128, 128)
devkit_cfg_pool = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_pool)

Network is valid


## Cropping

除此之外，可以选择将 DVS 输入范围进行剪裁（个人感觉没必要，128*128已经够小了）

对DVS输入区域进行裁剪，主要有两种方式：

input_shape初始化时修改参数DynapcnnNetwork。

修改硬件配置samna.speckxx.configuration.SpeckConfiguration.dvs_layer.origin和.cut。.origin定义DVS 数组左上角点的坐标。.cut定义DVS 数组右下点的坐标。

To crop the DVS's input region, there are mainly two kinds of ways:

1. Modify the `input_shape` argument when initialize the `DynapcnnNetwork` .

2. Modify the hardware configuration `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.origin` and `.cut`. `.origin` defines the coordinate of the **top-left** point of the DVS array. `.cut` defines the coordinate of the **bottom-right** point of the DVS array.

In [44]:
snn = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration that crop the input region to 32 x 32
input_shape = (1, 32, 32)
devkit_cfg_crop = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])

# only use the top-left 32 x 32 region of the DVS array
print(f"The top-left coordinate: {devkit_cfg_crop.dvs_layer.origin}")
print(f"The bottom-right coordinate: {devkit_cfg_crop.dvs_layer.cut}")

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_crop)

Network is valid
The top-left coordinate: (0,0)
The bottom-right coordinate: (31,31)


In [45]:
snn = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration that crop the input region to 64 x 64
input_shape = (1, 64, 64)
devkit_cfg_crop = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])

# use the central 64 x 64 region of the DVS array.
devkit_cfg_crop.dvs_layer.origin.x = 31
devkit_cfg_crop.dvs_layer.origin.y = 31
devkit_cfg_crop.dvs_layer.cut.x = 94
devkit_cfg_crop.dvs_layer.cut.y = 94

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_crop)

Network is valid


注意： 即使我们将硬件设置为使用 DVS 阵列的 64 x 64 中心区域，"DynapcnnVisualizer "也不会将裁剪区域移动到窗口中心！将 `.origin` 坐标从 (31, 31) 改为 (0, 0)，将 `.cut` 坐标从 (94, 94) 改为 (63, 63)。

## 镜像

只能通过修改 `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.mirror.x` 或 `.y` 或 `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.mirror_diagonal = True` 来设置镜像配置。 默认情况下，它们都被设置为 `False`。

In [46]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_mirror_x = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
# mirror the DVS events along with x-axis
devkit_cfg_mirror_x.dvs_layer.mirror.x = True

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_mirror_x)

Network is valid


In [47]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_mirror_diagonal = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
# mirror along with diagonal line
devkit_cfg_mirror_diagonal.dvs_layer.mirror_diagonal = True

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_mirror_diagonal)

Network is valid


## 禁止 DVS 事件进入处理器

在某些情况下，用户可能不希望将 DVS 事件发送到 DynapCNN Core，例如使用预先录制的事件运行硬件推理。

设置硬件配置 `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.pass_sensor_events = False` 可以实现这一目标。

In [48]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_not_pass_dvs = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
# do not let the DVS events be sent to processor
devkit_cfg_not_pass_dvs.dvs_layer.pass_sensor_events = False

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_not_pass_dvs)

Network is valid


## 输出目标层选择

 与 DynaCNN 核心/层一样，DVS 层也有两个输出目标层。

 通过修改 "samna.speckxx.configuration.SpeckConfiguration.dvs_layer.destinations"，可以选择将预处理后的 DVS 事件输出到哪两个层。

In [49]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_output_layer_select = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
for destination_index in [0, 1]:
    print(f"destination layer {destination_index} enable: {devkit_cfg_output_layer_select.dvs_layer.destinations[destination_index].enable}")
    print(f"destination layer {destination_index} to: {devkit_cfg_output_layer_select.dvs_layer.destinations[destination_index].layer }")

    
# not send the DVS events to layer #0
devkit_cfg_output_layer_select.dvs_layer.destinations[0].enable = False
devkit_cfg_output_layer_select.dvs_layer.destinations[0].layer = 0

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_output_layer_select)

Network is valid
destination layer 0 enable: True
destination layer 0 to: 0
destination layer 1 enable: False
destination layer 1 to: 0


## 开关极性。


硬件配置 `samna.speckxx.configuration.SpeckConfiguration.dvs_layer.on_channel` 和 `.off_channel` 控制 DVS 的两个极性。默认情况下，它们都被设置为 `True`。通过将它们设置为 `False`，可以关闭一个或两个通道。

In [42]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_switch_off_p = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
    
# switch off one channel
# try to set both channels to False to see what will happen!
devkit_cfg_switch_off_p.dvs_layer.on_channel = True
devkit_cfg_switch_off_p.dvs_layer.off_channel = False

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_switch_off_p)

Network is valid


## 热像素过滤

Speck 上的 DVS 层可选择对原始 DVS 事件应用过滤器。

通过修改 `samna.speckxx.configuration.SpeckConfiguration.dvs_filter`，用户可以对原始 DVS 事件应用热像素过滤器。有关过滤的更多详情，请参阅 [samna documentation.](https://synsense-sys-int.gitlab.io/samna/reference/speck2f/configuration/index.html#samna.speck2f.configuration.DvsFilterConfig)。

此外，如果要使用 DVS 过滤器块，需要为其提供外部慢时钟。通过调用 [samna's Unifirm Module](https://synsense-sys-int.gitlab.io/samna/reference/speck2fBoards/index.html#samna.speck2fBoards.UnifirmModule) 的 API，可以为 Speck 提供外部慢时钟。

In [50]:
snn = nn.Sequential(
    nn.Conv2d(2, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    IAFSqueeze(batch_size=1, min_v_mem=-1.0)
)

# make a new hardware configuration 
input_shape = (2, 128, 128)
devkit_cfg_filter = DynapcnnNetwork(snn=snn, input_shape=input_shape, dvs_input=True).make_config(device=devkit_name,
                                                                                                monitor_layers=["dvs", -1])
    
# set the filtering config
devkit_cfg_filter.dvs_filter.enable = True
devkit_cfg_filter.dvs_filter.filter_size.x = 3
devkit_cfg_filter.dvs_filter.filter_size.y = 3
devkit_cfg_filter.dvs_filter.hot_pixel_filter_enable = True
devkit_cfg_filter.dvs_filter.threshold = 5

# set up the Unifirm/IO module
devkit_io = devkit.get_io_module()

# update the configuration 
dynapcnn.samna_device.get_model().apply_configuration(devkit_cfg_filter)

Network is valid
